In [3]:
from langchain.tools import tool
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate
import json

client = OpenAI()

In [6]:
user_profile = """
John has 6 years of experience as a software engineer.
He has worked on backend systems using Python, Django, and Flask.
He has experience with cloud platforms like AWS and Docker.
He occasionally mentors junior developers.
"""

In [7]:
from langchain.tools import tool
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate
import json

client = OpenAI()

@tool
def analyze_experience(user_profile: str) -> dict:
    """
    Extract experience and seniority from a user profile.
    """
    prompt = f"""
    Analyze the user profile and extract experience information.

    USER PROFILE:
    {user_profile}

    Respond ONLY in JSON:
    {{
      "total_experience_years": <number>,
      "seniority_level": "<Junior / Mid / Senior>",
      "experience_breakdown": {{
        "technical": "<Low/Medium/High>",
        "leadership": "<Low/Medium/High>",
        "domain_specific": "<Low/Medium/High>"
      }}
    }}
    """

    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        max_output_tokens=300
    )

    return response.output_text

@tool
def analyze_competencies(user_profile: str) -> dict:
    """
    Identify core and secondary competencies from a user profile.
    """
    prompt = f"""
    Identify the core competencies of the following user profile.

    USER PROFILE:
    {user_profile}

    Respond ONLY in JSON:
    {{
      "core_competencies": ["Skill 1", "Skill 2"],
      "secondary_skills": ["Skill A", "Skill B"],
      "confidence": "<High / Medium / Low>"
    }}
    """

    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        max_output_tokens=300
    )

    return response.output_text
@tool
def recommend_roles(user_profile: str) -> dict:
    """
    Recommend suitable roles based on a user profile.
    """
    prompt = f"""
    Recommend suitable job roles for the following profile.

    USER PROFILE:
    {user_profile}

    Respond ONLY in JSON:
    {{
      "recommended_roles": [
        {{
          "role": "Role Name",
          "fit_reason": "Why this role fits",
          "readiness": "<Ready / Almost Ready / Needs Upskilling>"
        }}
      ]
    }}
    """

    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        max_output_tokens=300
    )

    return response.output_text

In [13]:
import json
import re

def format_json_output(raw_text: str) -> dict:
    # Remove markdown code fences
    cleaned = re.sub(r"```json|```", "", raw_text).strip()
    
    # Parse JSON
    parsed = json.loads(cleaned)
    
    # Pretty print (optional)
    #print(json.dumps(parsed, indent=2))
    
    return parsed





In [10]:
experience_output = analyze_experience.invoke(user_profile)
experience_output

'```json\n{\n  "total_experience_years": 6,\n  "seniority_level": "Senior",\n  "experience_breakdown": {\n    "technical": "High",\n    "leadership": "Medium",\n    "domain_specific": "Medium"\n  }\n}\n```'

In [14]:
formatted_output = format_json_output(experience_output)
formatted_output

{'total_experience_years': 6,
 'seniority_level': 'Senior',
 'experience_breakdown': {'technical': 'High',
  'leadership': 'Medium',
  'domain_specific': 'Medium'}}

### Rule Driven

In [15]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

intent_prompt = PromptTemplate.from_template("""
You are an intent classification system.

TASK:
Classify the user query into one or more intents.

INTENTS:
- experience
- competencies
- roles

USER QUERY:
{query}

RULES:
- Choose ONLY from the given intents
- Return JSON list of intents
- Do not add explanations

OUTPUT FORMAT:
{{
  "intents": ["experience", "roles"]
}}
""")

def decide_and_run_tools(user_query: str, user_profile: str):
    """
    Decide which tools to run based on user query and execute them.
    """

    # ---- Step 1: Detect Intent ----
    intent_response = llm.invoke(
        intent_prompt.format(query=user_query)
    )

    intents = json.loads(intent_response.content)["intents"]

    results = {}

    # ---- Step 2: Route to Tools ----
    if "experience" in intents:
        results["experience"] = analyze_experience.invoke(user_profile)

    if "competencies" in intents:
        results["competencies"] = analyze_competencies.invoke(user_profile)

    if "roles" in intents:
        results["roles"] = recommend_roles.invoke(user_profile)

    return results


In [21]:
# Example queries
query_1 = "How experienced is this candidate?"
query_2 = "What roles would suit this profile?"
query_3 = "Tell me the skills and suitable roles"

In [22]:
res_obj = decide_and_run_tools(query_1, user_profile)
print(res_obj)

formatted_output = format_json_output(res_obj['experience'])
print(formatted_output)

{'experience': '```json\n{\n  "total_experience_years": 6,\n  "seniority_level": "Senior",\n  "experience_breakdown": {\n    "technical": "High",\n    "leadership": "Medium",\n    "domain_specific": "Medium"\n  }\n}\n```'}
{'total_experience_years': 6, 'seniority_level': 'Senior', 'experience_breakdown': {'technical': 'High', 'leadership': 'Medium', 'domain_specific': 'Medium'}}


In [24]:
res_obj = decide_and_run_tools(query_2, user_profile)
print(res_obj)

formatted_output = format_json_output(res_obj['roles'])
print(formatted_output)

{'roles': '```json\n{\n  "recommended_roles": [\n    {\n      "role": "Senior Software Engineer",\n      "fit_reason": "John has 6 years of experience and strong skills in backend systems, making him well-suited for a senior role.",\n      "readiness": "Ready"\n    },\n    {\n      "role": "DevOps Engineer",\n      "fit_reason": "His experience with AWS and Docker aligns well with the requirements of a DevOps role.",\n      "readiness": "Ready"\n    },\n    {\n      "role": "Technical Lead",\n      "fit_reason": "His mentorship experience indicates potential for leadership in guiding teams and projects.",\n      "readiness": "Almost Ready"\n    },\n    {\n      "role": "Cloud Solutions Architect",\n      "fit_reason": "With his background in cloud platforms, John can design and implement cloud solutions effectively.",\n      "readiness": "Almost Ready"\n    },\n    {\n      "role": "Backend Architect",\n      "fit_reason": "His expertise in backend systems positions him well for archit

### use LLM to decide the Intent and then execute tools rule wise

In [ ]:
def decide_and_run_tools(user_query: str, user_profile: str):
    """
    Decide which tools to run based on user query and execute them.
    """

    # ---- Step 1: Detect Intent ----
    intent_response = llm.invoke(
        intent_prompt.format(query=user_query)
    )

    intents = json.loads(intent_response.content)["intents"]

    results = {}

    # ---- Step 2: Route to Tools ----
    if "experience" in intents:
        results["experience"] = analyze_experience.invoke(user_profile)

    if "competencies" in intents:
        results["competencies"] = analyze_competencies.invoke(user_profile)

    if "roles" in intents:
        results["roles"] = recommend_roles.invoke(user_profile)

    return results


In [26]:
print(decide_and_run_tools(query_1, user_profile))
print(decide_and_run_tools(query_2, user_profile))
print(decide_and_run_tools(query_3, user_profile))

{'experience': '```json\n{\n  "total_experience_years": 6,\n  "seniority_level": "Senior",\n  "experience_breakdown": {\n    "technical": "High",\n    "leadership": "Medium",\n    "domain_specific": "Medium"\n  }\n}\n```'}
{'roles': '```json\n{\n  "recommended_roles": [\n    {\n      "role": "Senior Software Engineer",\n      "fit_reason": "John has significant experience in backend systems and has mentorship experience, making him suitable for senior roles.",\n      "readiness": "Ready"\n    },\n    {\n      "role": "DevOps Engineer",\n      "fit_reason": "With his knowledge of AWS and Docker, John can effectively bridge development and operations roles.",\n      "readiness": "Almost Ready"\n    },\n    {\n      "role": "Technical Lead",\n      "fit_reason": "His mentoring experience indicates he can lead a team and guide junior developers effectively.",\n      "readiness": "Almost Ready"\n    },\n    {\n      "role": "Cloud Solutions Architect",\n      "fit_reason": "Experience with 

In [29]:
def decide_and_run_tools(user_query: str, user_profile: str):
    """
    Decide which tools to run based on user query and execute them.
    """

    # ---- Step 1: Detect Intent ----
    intent_response = llm.invoke(
        intent_prompt.format(query=user_query)
    )

    intents = json.loads(intent_response.content)["intents"]

    results = {}

    # ---- Step 2: Route to Tools ----
    if "experience" in intents:
        results["experience"] = analyze_experience.invoke(user_profile)
        formatted_output = format_json_output(results["experience"])

    if "competencies" in intents:
        results["competencies"] = analyze_competencies.invoke(user_profile)
        formatted_output = format_json_output(results["competencies"])

    if "roles" in intents:
        results["roles"] = recommend_roles.invoke(user_profile)
        formatted_output = format_json_output(results["roles"])

    return formatted_output


In [30]:
print(decide_and_run_tools(query_1, user_profile))
print(decide_and_run_tools(query_2, user_profile))
print(decide_and_run_tools(query_3, user_profile))

{'total_experience_years': 6, 'seniority_level': 'Senior', 'experience_breakdown': {'technical': 'High', 'leadership': 'Medium', 'domain_specific': 'Medium'}}
{'recommended_roles': [{'role': 'Senior Software Engineer', 'fit_reason': 'John has significant experience in backend development and mentoring, making him suitable for a senior role.', 'readiness': 'Ready'}, {'role': 'Cloud Engineer', 'fit_reason': 'His experience with AWS and Docker aligns well with cloud engineering responsibilities.', 'readiness': 'Almost Ready'}, {'role': 'Technical Lead', 'fit_reason': 'His mentoring experience indicates a capability to lead teams and oversee projects.', 'readiness': 'Almost Ready'}, {'role': 'DevOps Engineer', 'fit_reason': 'His familiarity with Docker and cloud platforms suggests a good fit for DevOps roles.', 'readiness': 'Almost Ready'}, {'role': 'Backend Architect', 'fit_reason': 'His expertise in backend systems positions him well for architectural design and decision making.', 'readi

### Use Agent System to Run Tools

In [39]:
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
import json
import re

# -------------------------
# LLM (SINGLE SOURCE)
# -------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# -------------------------
# Utility: Clean JSON
# -------------------------
def clean_json(text: str) -> dict:
    cleaned = re.sub(r"```json|```", "", text).strip()
    return json.loads(cleaned)

# -------------------------
# TOOLS
# -------------------------
@tool
def analyze_experience(user_profile: str) -> dict:
    """Extract experience and seniority from a user profile."""

    prompt = f"""
    Analyze the user profile and extract experience information.

    USER PROFILE:
    {user_profile}

    Respond ONLY in JSON:
    {{
      "total_experience_years": <number>,
      "seniority_level": "<Junior / Mid / Senior>",
      "experience_breakdown": {{
        "technical": "<Low/Medium/High>",
        "leadership": "<Low/Medium/High>",
        "domain_specific": "<Low/Medium/High>"
      }}
    }}
    """

    response = llm.invoke(prompt)
    return clean_json(response.content)


@tool
def analyze_competencies(user_profile: str) -> dict:
    """Identify core and secondary competencies."""

    prompt = f"""
    Identify the core competencies of the following user profile.

    USER PROFILE:
    {user_profile}

    Respond ONLY in JSON:
    {{
      "core_competencies": ["Skill 1", "Skill 2"],
      "secondary_skills": ["Skill A", "Skill B"],
      "confidence": "<High / Medium / Low>"
    }}
    """

    response = llm.invoke(prompt)
    return clean_json(response.content)


@tool
def recommend_roles(user_profile: str) -> dict:
    """Recommend suitable roles based on a user profile."""

    prompt = f"""
    Recommend suitable job roles for the following profile.

    USER PROFILE:
    {user_profile}

    Respond ONLY in JSON:
    {{
      "recommended_roles": [
        {{
          "role": "Role Name",
          "fit_reason": "Why this role fits",
          "readiness": "<Ready / Almost Ready / Needs Upskilling>"
        }}
      ]
    }}
    """

    response = llm.invoke(prompt)
    return clean_json(response.content)

# -------------------------
# AGENT
# -------------------------
agent = create_react_agent(
    model=llm,
    tools=[analyze_experience, analyze_competencies, recommend_roles],
)

resultObj = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": f"""
User query: What roles are suitable?
Profile:
{user_profile}
"""
        }
    ]
})

/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_10258/3226769732.py:101: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [ ]:
resultObj

{'messages': [HumanMessage(content='\nUser query: What roles are suitable?\nProfile:\n\nJohn has 6 years of experience as a software engineer.\nHe has worked on backend systems using Python, Django, and Flask.\nHe has experience with cloud platforms like AWS and Docker.\nHe occasionally mentors junior developers.\n\n', additional_kwargs={}, response_metadata={}, id='fd2d13a3-1666-4a9a-8bf3-c09f3d72744d'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 150, 'total_tokens': 208, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_11f3029f6b', 'id': 'chatcmpl-CmLXC0hPKxmAx7jbnDXeNyJnRebs8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprob

In [49]:
from langchain_core.messages import AIMessage

def get_final_answer(agent_result):
    """
    Extract the final user-facing answer from LangGraph output.
    """
    messages = agent_result["messages"]

    # Traverse in reverse to find final AI message
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content:
            return msg.content

    return None
final_answer = get_final_answer(resultObj)
print(final_answer)


Based on John's profile, here are some suitable roles along with the reasons for their fit and readiness levels:

1. **Senior Software Engineer**
   - **Fit Reason:** John has significant experience in backend development and mentoring, making him suitable for a senior position.
   - **Readiness:** Ready

2. **Backend Developer**
   - **Fit Reason:** His expertise in Python, Django, and Flask aligns well with backend development roles.
   - **Readiness:** Ready

3. **Cloud Engineer**
   - **Fit Reason:** Experience with AWS and Docker positions him well for roles focused on cloud infrastructure and services.
   - **Readiness:** Almost Ready

4. **Technical Lead**
   - **Fit Reason:** His mentoring experience suggests he could effectively lead a team of developers.
   - **Readiness:** Almost Ready

5. **DevOps Engineer**
   - **Fit Reason:** Familiarity with Docker and cloud platforms makes him a good candidate for DevOps roles.
   - **Readiness:** Needs Upskilling

These roles leverage